## 1. Summary

In the analysis below, I propose new "[Next Gen Stats](https://nextgenstats.nfl.com/glossary)" for pass defense, and a framework for evaluating whether these are useful statistics. These innovative stats can help us better understand individual player skill in specific aspects of pass defense (i.e. Target Prevention, Coverage, Reaction, Disruption, Penalty Avoidance), as well the ability of a player to impact the performance of teammates (i.e. ISLAND, BOOST). These new stats are all measured "over expectation," similar to [Completion Probability Over Expected (CPOE) or Rushing Yards Over Expected (RYOE)](https://www.nfl.com/news/next-gen-stats-intro-to-expected-rushing-yards). The statistics are based on how well a player performs relative to an "expected value," which is calculated from machine learning models that are automatically trained to estimate the mean expectation of a given situation and outcome.

One novel difference from prior work is that the "actual" statistic values are expectations. For example, rather than comparing a player’s actual value (i.e. completion percentage, rush yards, interceptions) to an expected value (i.e. expected completion percentage, expected rush yards, expected interceptions), instead we measure a players expected value in a given situation (i.e. expected defensive success rate at pass arrival) to an expected value at a prior timestamp (i.e. expected defensive success rate at pass throw). This allows us to improve metric stability by considering performance across a larger sample of plays (i.e. even when not targeted) and does not over weight rare statistics that exist for defensive players (e.g. the NFL leader in interceptions only had 6 in 2019). Additionally, by only considering the expected play outcome (instead of actual play EPA), this allows us to focus on what we are trying to measure (i.e. pass coverage), but not be biased by events that are largely luck (i.e. missed tackles, fumbles, defensive touchdowns).

This analysis concluded that proposed statistics **Pass Coverage Success Rate Over Expectation (PCOE)**, **Target Coverage Success Rate Over Expectation (TCOE)**, and **Target Reaction Success Rate Over Expectation (TROE)** all show positive signs in terms of "stickiness" and correlation with Defensive EPA. With some additional fine-tuning we believe they can be useful additions to NFL Next Gen Stats.

## 2. Background

Starting in 2015 the NFL [began embedding RFID tags in players shoulder pads](https://operations.nfl.com/the-game/technology/nfl-next-gen-stats/). This enabled the NFL to start capturing real-time data for every player on every play. This data has unlocked innovative [Next Gen Stats](https://nextgenstats.nfl.com/glossary) such as [Completion Probability Over Expected (CPOE) and Rushing Yards Over Expected (RYOE)](https://www.nfl.com/news/next-gen-stats-intro-to-expected-rushing-yards). Though metrics for analyzing quarterbacks, running backs, and wide receivers are consistently a part of public discourse, techniques for analyzing the defensive part of the game trail and lag behind. The goal of the [2021 NFL Big Data Bowl](https://www.kaggle.com/c/nfl-big-data-bowl-2021/overview) is to identify unique and impactful approaches to measure defensive performance on passing plays.

## 3. Objective

In the analysis below I will outline innovative metrics that can be used to evaluate which players are most successful at defending pass plays. The goal of this research is to identify new "Next Gen Stats" that are both stable and predictive of defensive success.


## 4. Methodology

### "Over Expected" Stats

Given the rising popularity of "Over Expected" stats in the NFL, my approach was to build on top of this proven technique. Such a method is powerful because it allows you to build cutting-edge machine learning models to best account for the complex player/team interactions that occur on the football field. The end result is statistics that are better able to control for context and situation when compared with traditional statistics. Even though the underlying process is sophisticated, the output is intuitive to understand (i.e. "performed X better/worse than the average expectation in that situation").

### Pass Defense Segments

Before proposing specific statistics, we must first define which segments of pass defense we do and don't want to measure. To start, here is one approach for how to break a pass play down into different components.

In [ ]:
import graphviz
graphviz.Source(
'digraph PassDefenseSegments {\
    graph [compound=true];\
    \
    subgraph cluster_0 {\
        label = "Pass Coverage Segments";\
        snap [label="Snap\n(ball_snap)"];\
        routes_ran [label="Routes Ran"];\
        pass [label="Pass Thrown\n(pass_forward)"];\
        arrived [label="Pass Arrived\n(pass_arrived)"];\
        pass_outcome [label="Pass Outcome\n(pass_outcome)"];\
        post [label="Subsequent events"];\
        play_outcome [label="Play Outcome"];\
        penalty [label="Penalty"];\
        play_success [label="Play Success"];\
    }\
    \
    snap -> routes_ran;\
    snap -> routes_ran;\
    snap -> routes_ran;\
    routes_ran -> pass [label="Target"];\
    pass -> arrived;\
    arrived -> pass_outcome;\
    pass_outcome -> post;\
    post -> play_outcome;\
    snap -> penalty [style=dashed];\
    penalty -> play_success [style=dashed];\
    play_outcome -> play_success;\
        \
    rankdir=TB;\
}')

Given that the goal of the Big Data Bowl is to "better understand the schemes and players that make for a successful defense against passing plays," we should focus on the events leading up to a pass outcome (i.e. complete, incomplete, interception, touchdown) and not subsequent events (e.g. yards after catch, missed tackles, forced fumbles, etc.).

While the ultimate goal is to better understand what contributes towards overall play success, we don't want our evaluation methods to be overly biased by events that happen after a pass outcome, as these should be considered other aspects of defense (i.e. tackling, ability to force fumbles, etc.). In the next section you will see how our approach effectively nets out post pass outcome variance by assuming an expected play outcome based on the pass outcome end state.

### Defensive-Success-Rate

Traditional pass defense statistics (such as interceptions or pass breakups) are noisy in that they measure infrequent events that have a large element of luck to them. Therefore, my approach was to shy away from metrics such as "interceptions over expectation," as this would likely result in stability issues where players performance would vary greatly year to year. 

One approach considered was to create metrics that intuitively correlate with pass coverage skill, such as "lack of receiver separation over expectation." While likely informative, such metrics may not always correlate with actual success and risk over-emphasizing the wrong behavior. For example, there are situations where it is optimal to give a receiver separation.

For these reasons, we chose to focus on measuring a player’s impact on overall play success (i.e. "Defensive-Success-Rate").

> **Defensive-Success-Rate (DSR)** - The rate at which plays are "won" by the defense. A play is considered a win by the defense if it resulted in negative EPA.

### "Expected" Models

To evaluate how a player performs relative to expected, I built numerous "Expected" machine learning models to estimate the mean expectation of given situation and outcome. These models were built using [AutoGluon](https://github.com/awslabs/autogluon), which is an AutoML framework which [succeeds by ensembling multiple models](https://arxiv.org/pdf/2003.06505.pdf) (i.e. gradient boosted trees, neural networks, etc.) and stacking them in multiple layers, all without human tuning.

The models were all trained on 78,333 "route" observations representing each receiver route classified in the provided data. Some of the models used all receiver routes, regardless of whether the receiver was targeted, whereas others were limited to targeted routes only (16,698).

All models were trained on a base set of features spanning situation, personnel, pass details, receiver route details, and receiver movement metrics. Additionally, some models had features for the closest defender (i.e. separation, positioning, location, movement), the football (i.e. location, movement), and the receiver (i.e. location, movement) all based on critical points of the play.

A complete list of features can be seen in Appendix A.1, feature importance for each model in Appendix A.2, and model evaluation (i.e. ROC curve, AUC) in Appendix A.3.

***Models***

1. **Pass-Thrown-No-Defender - Expected Defensive-Success-Rate** (no defender features) - The probability of a targeted pass ending in success for the defense based on numerous factors known at the time of throw.
1. **Pass-Thrown - Expected Defensive-Success-Rate** - The probability of a targeted pass ending in success for the defense based on numerous factors known at the time of throw.
1. **Pass-Arrived - Expected Defensive-Success-Rate** - The probability of a targeted pass ending in success for the defense based on numerous factors known at the time of pass arrival.
1. **Pass-Finished - Expected Defensive-Success-Rate** - The probability of a targeted pass ending in success for the defense based on numerous factors known at the time of pass outcome.
1. **Expected Target Percentage** (no defender features) - The probability of a receiver route being targeted based on numerous factors known at the time of throw.
1. **Expected Penalty Percentage** (no defender features) - The probability of the closest defender being called for a penalty based on numerous factors known at the time of throw.
1. **Expected Closest Help Defender Yards** (no defender features) - The expected yards between the *second* closest defender and the receiver being targeted.

See Appendix A.4 for more detailed model considerations.

In [ ]:
graphviz.Source(
'digraph Models {\
    graph [compound=true];\
	subgraph cluster_0 {\
		label = "Pass Coverage Segments";\
        snap [label="Snap\n(ball_snap)"];\
        routes_ran [label="Routes Ran"];\
        pass [label="Pass Thrown\n(pass_forward)"];\
        arrived [label="Pass Arrived\n(pass_arrived)"];\
        pass_outcome [label="Pass Outcome\n(pass_outcome)"];\
        post [label="Subsequent events", color=lightgrey, fontcolor=lightgrey];\
        play_outcome [label="Play Outcome", color=lightgrey, fontcolor=lightgrey];\
        penalty [label="Penalty", color=lightgrey, fontcolor=lightgrey];\
        play_success [label="Play Success", color=lightgrey, fontcolor=lightgrey];\
	}\
\
    snap -> routes_ran;\
    snap -> routes_ran;\
    snap -> routes_ran;\
    routes_ran -> pass [label="Target"];\
    pass -> arrived;\
    arrived -> pass_outcome;\
    pass_outcome -> post [color=lightgrey];\
    post -> play_outcome [color=lightgrey];\
    snap -> penalty [color=lightgrey, style=dashed];\
    penalty -> play_success [color=lightgrey, style=dashed];\
    play_outcome -> play_success [color=lightgrey];\
\
    subgraph cluster_1 {\
    	label = "Expected Defensive-Success-Rate Models";\
        rr_sr [label="Pass-Thrown-No-Defender"];\
        pc_sr [label="Pass-Thrown"];\
        pa_sr [label="Pass-Arrived"];\
        pcon_sr [label="Pass-Finished"];\
    \
        rr_sr -> pc_sr [color="transparent"];\
        pc_sr -> pa_sr [color="transparent"];\
        pa_sr -> pcon_sr [color="transparent"];\
    }\
\
    pcon_sr -> play_success [label="Objective", style="dashed" ltail=cluster_1, color=lightgrey, fontcolor=lightgrey]; \
\
    subgraph cluster_2 {\
    	label = "Other Models";\
        pp_sr [label="Penalty Percentage"];\
        tp_sr [label="Target Percentage"];\
        ch_sr [label="Closest Help Defender Yards"];\
    \
        pp_sr -> tp_sr [color="transparent"];\
        tp_sr -> ch_sr [color="transparent"];\
    }\
    pass -> pp_sr [lhead=cluster_2]; \
    \
    pass -> rr_sr;\
    pass -> pc_sr;\
    arrived -> pa_sr;\
    pass_outcome -> pcon_sr;\
    rankdir=TB;\
}')

### Next Gen Stat Definitions

The following statistics are based comparisons between expected defensive-success-rate across various critical points of a play. A player is evaluated anytime they are the closest defender to a receiver at the time a pass was thrown (or a sack occurred). Some statistics consider all routes defended even if the receiver was not thrown to, whereas other statistics only consider routes where the receiver was targeted (noted in definition as "all plays" or "targets only"). We are making the assumption that the closest defender at the time of pass is typically *most* responsible for guarding a given receiver on a play.

 1. **Pass Coverage Success Rate Over Expectation (PCOE)** - Expected Defensive-Success-Rate at Pass-Thrown compared to Pass-Thrown-No-Defender (all plays)
    
 2. **Target Coverage Success Rate Over Expectation (TCOE)** - Expected Defensive-Success-Rate at Pass-Thrown compared to Pass-Thrown-No-Defender (targets only)

 3. **Target Reaction Success Rate Over Expectation (TROE)** - Expected Defensive-Success-Rate at Pass-Arrived compared to at Pass-Thrown (targets only)

 4. **Target Disruption Success Rate Over Expectation (TDOE)** - Expected Defensive-Success-Rate at Pass-Finished compared to at Pass-Arrived (targets only)

One downside with the statistics above (only assign blame/credit to the closest defender) is that they don't properly capture team dynamics. For example, on a blown coverage the closest defender at time of throw will take the blame even if they aren't the one who made the mistake.

Only looking at the closest defender also overlooks "halo effects" where defenders can strengthen the coverage of others around them (e.g. effectively handing off receivers in zone coverage, allowing others to play more aggressively as they know help is nearby, shutdown corners requiring less help, etc.).

To help minimize these concerns, the following statistics were created based on the *second* closest defender to a receiver at the time of pass.

 5. **Closest Help Defender Yards Over Expectation (CHOE) = Isolation Success Leads A Nice Defense (ISLAND)** - Actual Closest Help Defender Yards compared to Expected Closest Help Defender Yards (all plays)

 6. **Target Coverage Help Over Expectation (TCHOE) = Benefit Others Or Strengthen Team (BOOST)** - Expected Defensive-Success-Rate at Pass-Thrown compared to at Pass-Thrown-No-Defender, after adjusting for the closest defenders TCOE (targets only)

The following statistics were also explored in order to capture additional specific factors not captured above.

 7. **Target Prevention Percentage Over Expectation (TPOE)** - Actual Target Percentage compared to Expected Target Percentage (all plays)

 8. **Penalty Avoidance Percentage Over Expectation (PAOE)** - Actual Penalty Avoidance Percentage compared to Expected Penalty Avoidance Percentage (all plays)


In [ ]:
graphviz.Source(
'digraph NextGenStats {\
    graph [compound=true];\
    \
    subgraph cluster_0 {\
        label = "Pass Coverage Segments";\
        snap [label="Snap\n(ball_snap)"];\
        routes_ran [label="Routes Ran"];\
        pass [label="Pass Thrown\n(pass_forward)"];\
        arrived [label="Pass Arrived\n(pass_arrived)"];\
        pass_outcome [label="Pass Outcome\n(pass_outcome)"];\
    }\
    \
    snap -> routes_ran;\
    snap -> routes_ran;\
    snap -> routes_ran;\
    routes_ran -> pass [label="Target"];\
    pass -> arrived;\
    arrived -> pass_outcome;\
    subgraph cluster_1 {\
        label = "Next Gen Stats\n(closest defender)";\
        tp [label="Target Prevention (TPOE)"];\
        pc [label="Pass Coverage (PCOE)"];\
        tc [label="Target Coverage (TCOE)"];\
        tr [label="Target Reaction (TROE)"];\
        td [label="Target Disruption (TDOE)"];\
        pa [label="Penalty Avoidance (PAOE)"];\
    \
        pa -> tp [color="transparent"];\
        tp -> pc [color="transparent"];\
        pc -> tc [color="transparent"];\
        tc -> tr [color="transparent"];\
        tr -> td [color="transparent"];\
    }\
    subgraph cluster_2 {\
        label = "Next Gen Stats\n(second closest defender)";\
        tch [label="Benefit Others Or Strengthen Team (BOOST)"];\
        ch [label="Isolation Success Leads A Nice Defense (ISLAND)"];\
        ch -> tch [color="transparent"];\
    }\
    \
    snap -> pa;\
    routes_ran -> tp;\
    routes_ran -> pc;\
    routes_ran -> tc;\
    tc -> ch;\
    pass -> tr;\
    tc -> tch;\
    arrived -> td;\
    rankdir=TB;\
}')

### Evaluation

The goal of this research is to identify new "Next Gen Stats" that are both stable and predictive of defensive success. To evaluate whether these new statistics are successful at achieving this goal, [one approach](https://hockey-graphs.com/2017/12/01/behind-the-numbers-what-makes-a-stat-good) is to formally evaluate the following criteria:

1. Correlation
 * *Does the stat correlate with play success?*
1. Autocorrelation
 * *Does the stat correlate with itself across sub-samples (i.e. is a player's value "sticky")?*

To perform these evaluations, I split the data into two sub-samples (1H, 2H) based on whether the play occurred in the first or second half of the season (weeks 1->9 and 10->17). Using TCOE as an example, I then calculated the following correlation coefficients to evaluate metric "predictiveness" and "stickiness".
1. TCOE vs. EPA (across plays)
1. TCOE from 1H vs. TCOE from 2H (across players)

*Note: While the models should be fairly robust, we use these same 1H/2H sub-samples to ensure predictions used to generate "expected" values were always out-of-sample.*

## 5. Results

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', delimiter=',')
plays['id'] = (plays['gameId'].astype(str) + plays['playId'].astype(str)).astype(float)

targeted = pd.read_csv("../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv", dtype={'targetNflId': np.float32})

games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv', delimiter=',')
games['sample'] = np.where(games['week']<=9, '1H', '2H')

players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv', delimiter=',', dtype={'nflId': np.float32})

tracking_data = pd.concat([pd.read_csv('../input/nfl-big-data-bowl-2021/week%s.csv'%w,
                                       delimiter=',', 
                                       dtype= {'jerseyNumber':np.float32,'x': np.float32,'y': np.float32,
                                               's': np.float32,'a': np.float32,
                                               'dir': np.float32,'o': np.float32,'nflId': np.float32},
                                       usecols=['x','y','s','a','o','dir',
                                                'event','nflId','jerseyNumber','frameId',
                                                               'team','gameId','playId','playDirection','route']) for w in range(1,18)])

tracking_data['id'] = (tracking_data['gameId'].astype(str) + tracking_data['playId'].astype(str)).astype(float)
tracking_data['playDirection'] = np.where(tracking_data['playDirection']=='left', 1, 0)

# flip
tracking_data['x'] = np.where(tracking_data['playDirection']==1, 120-tracking_data['x'], tracking_data['x'])
tracking_data['y'] = np.where(tracking_data['playDirection']==1, 160/3-tracking_data['y'], tracking_data['y'])

# 0 degrees: the offensive player is moving completely to his left
# 90 degrees: the offensive player is moving straight ahead, towards opponent end zone
# 180 degrees: the offensive player is moving completely to his right
# 270 degrees: the offensive player is moving backwards, towards his own team's end zone (this is generally bad)

#   mutate(Dir_std_1 = ifelse(ToLeft & Dir < 90, Dir + 360, Dir), 
#          Dir_std_1 = ifelse(!ToLeft & Dir > 270, Dir - 360, Dir_std_1))
#           mutate(Dir_std_2 = ifelse(ToLeft, Dir_std_1 - 180, Dir_std_1))
tracking_data['o_std'] = np.where((tracking_data['playDirection']==1)&(tracking_data['o'] < 90),
                                    tracking_data['o']+360, tracking_data['o'])
tracking_data['o_std'] = np.where((tracking_data['playDirection']==0)&(tracking_data['o'] > 270),
                                    tracking_data['o']-360, tracking_data['o_std'])
tracking_data['o_std'] = np.where(tracking_data['playDirection']==1, tracking_data['o_std']-180, tracking_data['o_std'])

tracking_data['dir_std'] = np.where((tracking_data['playDirection']==1)&(tracking_data['dir'] < 90),
                                    tracking_data['dir']+360, tracking_data['dir'])
tracking_data['dir_std'] = np.where((tracking_data['playDirection']==0)&(tracking_data['dir'] > 270),
                                    tracking_data['dir']-360, tracking_data['dir_std'])
tracking_data['dir_std'] = np.where(tracking_data['playDirection']==1, tracking_data['dir_std']-180, tracking_data['dir_std'])

tracking_data['o'] = tracking_data['o_std']
tracking_data['dir'] = tracking_data['dir_std']

tracking_data = tracking_data.drop('o_std',axis=1)
tracking_data = tracking_data.drop('dir_std',axis=1)

ball_snap = tracking_data.query('event=="ball_snap"').groupby(['id'])['frameId'].min()
pass_thrown_or_sack = tracking_data[tracking_data.event.isin(['pass_forward','pass_shovel','pass_lateral','qb_sack','qb_strip_sack','safety'])].groupby(['id'])['frameId'].min()
pass_arrived = tracking_data.query('event=="pass_arrived"').groupby(['id'])['frameId'].min()
pass_outcome = tracking_data[tracking_data.event.str.startswith('pass_outcome')].groupby(['id'])['frameId'].min()

tracking_data['ball_snap'] = tracking_data[['id']].merge(ball_snap, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
tracking_data['pass_thrown_or_sack'] = tracking_data[['id']].merge(pass_thrown_or_sack, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
tracking_data['pass_arrived'] = tracking_data[['id']].merge(pass_arrived, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
tracking_data['pass_outcome'] = tracking_data[['id']].merge(pass_outcome, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values

plays['ball_snap'] = plays.merge(ball_snap, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
plays['pass_thrown_or_sack'] = plays.merge(pass_thrown_or_sack, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
plays['pass_arrived'] = plays.merge(pass_arrived, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values
plays['pass_outcome'] = plays.merge(pass_outcome, how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one')['frameId'].values

print(len(plays))

# drop QB spikes
plays = plays[~plays.id.isin(tracking_data[tracking_data.event.isin(['qb_spike'])].id.drop_duplicates())]
print(len(plays))

# drop when there was no sack or throw
plays = plays[~plays.pass_thrown_or_sack.isnull()]
print(len(plays))

plays['dsr'] = np.where(plays['epa']<0, 1, 0)

plays.info()

routes = tracking_data.dropna(subset=['route'])[['id','gameId','playId','nflId','route']].drop_duplicates(subset=['id','nflId'])

routes = routes.merge(plays[['id', 'down', 'yardsToGo',
                             'offenseFormation', 'defendersInTheBox','numberOfPassRushers',
                             'absoluteYardlineNumber','penaltyJerseyNumbers',
#        'possessionTeam', 'playType', 'yardlineSide', 'yardlineNumber',
#        'offenseFormation', 'personnelO', 'defendersInTheBox',
#        'numberOfPassRushers', 'personnelD', 'typeDropback',
#        'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
#        'absoluteYardlineNumber', 'penaltyCodes', 'penaltyJerseyNumbers',
#        'passResult', 'offensePlayResult', 'playResult', 'isDefensivePI',
#        'ball_snap', 'pass_thrown', 
#        'qb_sack',
                             'epa','dsr',
    'ball_snap','pass_thrown_or_sack','pass_arrived', 'pass_outcome']], how='left',left_on=['id'],right_on=['id'],
                    validate='many_to_one').dropna(subset=['pass_thrown_or_sack'])

routes['targeted'] = routes.merge(targeted, how='left',left_on=['gameId','playId'],right_on=['gameId','playId'], validate='many_to_one')['targetNflId'].values
routes['targeted'] = np.where(routes['nflId']==routes['targeted'], 1, 0)
routes['route_id'] = routes.index

routes['time_to_pass'] = routes['pass_thrown_or_sack'] - routes['ball_snap']

routes['sample'] = routes.merge(games[['gameId','sample']], how='left',left_on=['gameId'],right_on=['gameId'], validate='many_to_one')['sample'].values

for c in ['x','y','s','a','o','dir']:
    routes['%s_wr_throw'%c] = routes.merge(tracking_data[tracking_data.frameId==tracking_data.pass_thrown_or_sack].drop_duplicates(), how='left',left_on=['id','nflId'],right_on=['id','nflId'], validate='many_to_one')[c].values
    routes['%s_wr_snap'%c] = routes.merge(tracking_data[tracking_data.frameId==tracking_data.ball_snap].drop_duplicates(), how='left',left_on=['id','nflId'],right_on=['id','nflId'], validate='many_to_one')[c].values
    routes['%s_wr_arrival'%c] = routes.merge(tracking_data[tracking_data.frameId==tracking_data.pass_arrived].drop_duplicates(), how='left',left_on=['id','nflId'],right_on=['id','nflId'], validate='many_to_one')[c].values
    routes['%s_wr_outcome'%c] = routes.merge(tracking_data[tracking_data.frameId==tracking_data.pass_outcome].drop_duplicates(), how='left',left_on=['id','nflId'],right_on=['id','nflId'], validate='many_to_one')[c].values

routes['wr_distance'] = np.sqrt(np.power(routes['x_wr_throw'] - routes['x_wr_snap'], 2) + np.power(routes['y_wr_throw'] - routes['y_wr_snap'], 2))

routes['route_team'] = routes.merge(tracking_data[tracking_data.frameId==tracking_data.pass_thrown_or_sack].drop_duplicates(), how='left',left_on=['id','nflId'],right_on=['id','nflId'], validate='many_to_one')['team'].values
routes['def_team'] = np.where(routes['route_team'] == 'away', 'home', 'away')

routes['home_team'] = routes[['gameId']].merge(games, how='left',left_on=['gameId'],right_on=['gameId'],validate='many_to_one')['homeTeamAbbr'].values
routes['away_team'] = routes[['gameId']].merge(games, how='left',left_on=['gameId'],right_on=['gameId'],validate='many_to_one')['visitorTeamAbbr'].values
routes['def_team_name'] = np.where(routes['def_team']=='home', routes['home_team'], routes['away_team'])

distances = routes[['route_id','id','def_team','x_wr_throw','y_wr_throw']].merge(
    tracking_data[tracking_data.frameId==tracking_data.pass_thrown_or_sack][['id','team','nflId','x','y','jerseyNumber']],
        how='left',left_on=['id','def_team'],right_on=['id','team'],validate='many_to_many').dropna()
distances['dist'] = np.sqrt(np.power(distances['x'] - distances['x_wr_throw'], 2) + np.power(distances['y'] - distances['y_wr_throw'], 2))
distances['distance_rank'] = distances.groupby('route_id')['dist'].rank(method='first').astype(int)

closest_defender = distances[distances.distance_rank==1][['route_id','nflId','jerseyNumber']]
second_closest_defender = distances[distances.distance_rank==2][['route_id','nflId']]

routes['closest_defender'] = routes[['route_id']].merge(closest_defender, how='left',left_on=['route_id'],right_on=['route_id'],validate='many_to_one')['nflId'].values
routes['closest_defender_jersey'] = routes[['route_id']].merge(closest_defender, how='left',left_on=['route_id'],right_on=['route_id'],validate='many_to_one')['jerseyNumber'].values
routes['second_closest_defender'] = routes[['route_id']].merge(second_closest_defender, how='left',left_on=['route_id'],right_on=['route_id'],validate='many_to_one')['nflId'].values

for c in ['x','y','s','a','o','dir']:
    routes['%s_cd_snap'%c] = routes[['id','closest_defender']].merge(
                tracking_data[tracking_data.frameId==tracking_data.ball_snap][['id','nflId',c]].dropna(subset=['nflId']).drop_duplicates(),
                                            how='left', left_on=['id','closest_defender'],right_on=['id','nflId'],
                                            validate='many_to_one')[c].values
    routes['%s_cd_pass'%c] = routes[['id','closest_defender']].merge(
                tracking_data[tracking_data.frameId==tracking_data.pass_thrown_or_sack][['id','nflId',c]].dropna(subset=['nflId']).drop_duplicates(),
                                                how='left', left_on=['id','closest_defender'],right_on=['id','nflId'],
                                                validate='many_to_one')[c].values
    routes['%s_cd_arrival'%c] = routes[['id','closest_defender']].merge(
                tracking_data[tracking_data.frameId==tracking_data.pass_arrived][['id','nflId',c]].dropna(subset=['nflId']).drop_duplicates(),
                                            how='left', left_on=['id','closest_defender'],right_on=['id','nflId'],
                                            validate='many_to_one')[c].values
    routes['%s_cd_outcome'%c] = routes[['id','closest_defender']].merge(
                tracking_data[tracking_data.frameId==tracking_data.pass_outcome][['id','nflId',c]].dropna(subset=['nflId']).drop_duplicates(),
                                            how='left', left_on=['id','closest_defender'],right_on=['id','nflId'],
                                            validate='many_to_one')[c].values
    routes['%s_scd_pass'%c] = routes[['id','second_closest_defender']].merge(
                tracking_data[tracking_data.frameId==tracking_data.pass_thrown_or_sack][['id','nflId',c]].dropna(subset=['nflId']).drop_duplicates(),
                                                how='left', left_on=['id','second_closest_defender'],right_on=['id','nflId'],
                                                validate='many_to_one')[c].values

routes['closest_distance_pass'] = np.sqrt(np.power(routes['x_wr_throw'] - routes['x_cd_pass'], 2) + np.power(routes['y_wr_throw'] - routes['y_cd_pass'], 2))
routes['second_closest_distance_pass'] = np.sqrt(np.power(routes['x_wr_throw'] - routes['x_scd_pass'], 2) + np.power(routes['y_wr_throw'] - routes['y_scd_pass'], 2))
routes['closest_distance_arrival'] = np.sqrt(np.power(routes['x_wr_arrival'] - routes['x_cd_arrival'], 2) + np.power(routes['y_wr_arrival'] - routes['y_cd_arrival'], 2))
routes['closest_distance_outcome'] = np.sqrt(np.power(routes['x_wr_outcome'] - routes['x_cd_outcome'], 2) + np.power(routes['y_wr_outcome'] - routes['y_cd_outcome'], 2))

# routes.head().T.tail(40)

for c in ['x','y','s','a']:
    routes['%s_football_snap'%c] = routes[['id']].merge(
        tracking_data[(tracking_data.frameId==tracking_data.ball_snap)&(tracking_data.team=='football')],
        how='left',left_on=['id'],right_on=['id'],validate='many_to_one')[c].values

    routes['%s_football_throw'%c] = routes[['id']].merge(
        tracking_data[(tracking_data.frameId==tracking_data.pass_thrown_or_sack)&(tracking_data.team=='football')],
        how='left',left_on=['id'],right_on=['id'],validate='many_to_one')[c].values

    routes['%s_football_arrival'%c] = routes[['id']].merge(
        tracking_data[(tracking_data.frameId==tracking_data.pass_arrived)&(tracking_data.team=='football')],
        how='left',left_on=['id'],right_on=['id'],validate='many_to_one')[c].values

    routes['%s_football_outcome'%c] = routes[['id']].merge(
        tracking_data[(tracking_data.frameId==tracking_data.pass_outcome)&(tracking_data.team=='football')],
        how='left',left_on=['id'],right_on=['id'],validate='many_to_one')[c].values

routes['air_yards'] = np.sqrt(np.power(routes['x_wr_throw'] - routes['x_football_throw'], 2) + np.power(routes['y_wr_throw'] - routes['y_football_throw'], 2))

routes['route_pos'] = routes[['nflId']].merge(players,how='left',left_on=['nflId'],right_on=['nflId'],validate='many_to_one')['position'].values

num_routes = routes.groupby(['id'])['nflId'].count().reset_index()
routes['num_routes'] = routes[['id']].merge(num_routes,how='left',left_on=['id'],right_on=['id'],validate='many_to_one')['nflId'].values
routes['is_home'] = np.where(routes.def_team=='home', 1, 0)

max_speed_route = tracking_data[(tracking_data.frameId>=tracking_data.ball_snap)&(
                                 tracking_data.frameId<=tracking_data.pass_thrown_or_sack)].groupby(['id','nflId'])['s'].max().to_frame().reset_index()
avg_speed_route = tracking_data[(tracking_data.frameId>=tracking_data.ball_snap)&(
                                 tracking_data.frameId<=tracking_data.pass_thrown_or_sack)].groupby(['id','nflId'])['s'].mean().to_frame().reset_index()
max_acc_route = tracking_data[(tracking_data.frameId>=tracking_data.ball_snap)&(
                                 tracking_data.frameId<=tracking_data.pass_thrown_or_sack)].groupby(['id','nflId'])['a'].max().to_frame().reset_index()

routes['max_speed_cd_route'] = routes[['id','closest_defender']].merge(max_speed_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['s'].values
routes['avg_speed_cd_route'] = routes[['id','closest_defender']].merge(avg_speed_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['s'].values
routes['max_acc_cd_route'] = routes[['id','closest_defender']].merge(max_acc_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['a'].values

routes['max_speed_wr_route'] = routes[['id','closest_defender']].merge(max_speed_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['s'].values
routes['avg_speed_wr_route'] = routes[['id','closest_defender']].merge(avg_speed_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['s'].values
routes['max_acc_wr_route'] = routes[['id','closest_defender']].merge(max_acc_route,how='left', left_on=['id','closest_defender'],right_on=['id','nflId'], validate='many_to_one')['a'].values

routes['penalty'] = routes['def_team_name'] + ' ' + routes['closest_defender_jersey'].fillna(-1).astype(int).astype(str)
routes['penalty'] = [x[0] in x[1] if x[0] is not None else False for x in zip(routes['penalty'].fillna('-1'), routes['penaltyJerseyNumbers'].fillna('-1'))]

# Install AutoGluon - https://www.kaggle.com/sgdread/mlcourse-ai-fall-2019-autogluon-starter/
!pip install -U pip setuptools wheel
!pip uninstall typing -y
!pip install -U mxnet
!pip install autogluon.tabular

import autogluon.core as ag
from autogluon.tabular import TabularPrediction as task

base_features = [
#     Quarterback play details
#     'typeDropback',
    'time_to_pass',
    'x_football_throw','y_football_throw', # Location at time of throw
# Alignment (reciever location at snap, defenders in the box)
    'offenseFormation', 'defendersInTheBox','numberOfPassRushers',
    'num_routes',
# Situation (field position, score, clock, down, yards to go)
    'down', 'yardsToGo',
    'absoluteYardlineNumber',
    'is_home',
# Receiver route details
    'route','route_pos',
    'x_wr_snap','y_wr_snap', # Location on field at snap
    'air_yards',
# Reciever movement metrics (speed, acceleration, distance traveled, yards downfield)
    'x_wr_throw','y_wr_throw',
#     Speed Max: Maximum speed of player on a play (yards/sec)
    'max_speed_wr_route','avg_speed_wr_route','max_acc_wr_route',
#     Speed Avg: Average speed of player on a play (yards/sec)
#     Acceleration Max: Maximum acceleration of player (yd/s²)
#     Total Distance: Distance traveled on the play from snap to pass event (yards)
    'wr_distance',
]

defender_throw_features = [
# Defender movement/positioning metrics
     's_wr_throw','a_wr_throw', 'o_wr_throw','dir_wr_throw', # Location on field at time of pass
    'x_cd_pass','y_cd_pass', 's_cd_pass','a_cd_pass', 'o_cd_pass','dir_cd_pass',
    'max_speed_cd_route','avg_speed_cd_route','max_acc_cd_route',
    'closest_distance_pass',
]

defender_arrival_features = [
#     'x_football_arrival','y_football_arrival', 's_football_arrival','a_football_arrival', 
    'x_wr_arrival','y_wr_arrival', 's_wr_arrival','a_wr_arrival', 'o_wr_arrival','dir_wr_arrival',
    'x_cd_arrival','y_cd_arrival', 's_cd_arrival','a_cd_arrival', 'o_cd_arrival','dir_cd_arrival',
    'closest_distance_arrival',
]

defender_outcome_features = [
#     'x_football_outcome','y_football_outcome', 's_football_outcome','a_football_outcome', 
    'x_wr_outcome','y_wr_outcome', 's_wr_outcome','a_wr_outcome', 'o_wr_outcome','dir_wr_outcome',
    'x_cd_outcome','y_cd_outcome', 's_cd_outcome','a_cd_outcome', 'o_cd_outcome','dir_cd_outcome',
    'closest_distance_outcome',
]

dv = 'dsr'

routes[base_features+[dv]].info()
routes[defender_throw_features].info()
routes[defender_arrival_features].info()
routes[defender_outcome_features].info()

time_run = 30

train_1h = routes.query('sample=="1H" and targeted==1')
train_2h = routes.query('sample=="2H" and targeted==1')
train_1h_all = routes.query('sample=="1H"')
train_2h_all = routes.query('sample=="2H"')

eval_metric = 'roc_auc'
presets = ['high_quality_fast_inference_only_refit','optimize_for_deployment']
excluded_model_types = ['RF','KNN','XT']

model_throw_no_defender_1h = task.fit(train_data=train_1h[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb1')
model_throw_no_defender_2h = task.fit(train_data=train_2h[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb2')

model_throw_1h = task.fit(train_data=train_1h[base_features+defender_throw_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb3')
model_throw_2h = task.fit(train_data=train_2h[base_features+defender_throw_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb4')

model_arrive_1h = task.fit(train_data=train_1h[base_features+defender_throw_features+defender_arrival_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb5')
model_arrive_2h = task.fit(train_data=train_2h[base_features+defender_throw_features+defender_arrival_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb6')

model_outcome_1h = task.fit(train_data=train_1h[base_features+defender_throw_features+defender_arrival_features+defender_outcome_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb7')
model_outcome_2h = task.fit(train_data=train_2h[base_features+defender_throw_features+defender_arrival_features+defender_outcome_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb8')

time_run = 30

dv = 'targeted'

model_target_1h = task.fit(train_data=train_1h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb9')
model_target_2h = task.fit(train_data=train_2h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb10')

dv = 'penalty'
model_penalty_1h = task.fit(train_data=train_1h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb11')
model_penalty_2h = task.fit(train_data=train_2h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, eval_metric=eval_metric, presets=presets, output_directory = '/tmp/autogluon/bdb12')

dv = 'second_closest_distance_pass'
model_island_1h = task.fit(train_data=train_1h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types, presets=presets, output_directory = '/tmp/autogluon/bdb13')
model_island_2h = task.fit(train_data=train_2h_all[base_features+[dv]].dropna(), label=dv, time_limits=60*time_run, excluded_model_types = excluded_model_types,  presets=presets, output_directory = '/tmp/autogluon/bdb14')

# Pass Coverage Success Rate Over Expectation (PCOE) - Expected Defensive-Success-Rate at Pass-Thrown compared to Pass-Thrown-No-Defender (all plays)
# Target Coverage Success Rate Over Expectation (TCOE) - Expected Defensive-Success-Rate at Pass-Thrown compared to Pass-Thrown-No-Defender (targets only)
# Target Reaction Success Rate Over Expectation (TROE) - Expected Defensive-Success-Rate at Pass-Arrived compared to at Pass-Thrown (targets only)
# Target Disruption Success Rate Over Expectation (TDOE) - Expected Defensive-Success-Rate at Pass-Finished compared to at Pass-Arrived (targets only)
# Isolation Success Leads A Nice Defense (ISLAND) - Actual Closest Help Defender Yards compared to Expected Closest Help Defender Yards (all plays)
# Benefit Others Or Strengthen Team (BOOST) - Expected Defensive-Success-Rate at Pass-Thrown compared to at Pass-Thrown-No-Defender, after adjusting for the closest defenders TCOE (targets only)
# Penalty Avoidance Percentage Over Expectation (PAOE) - Actual Penalty Avoidance Percentage compared to Expected Penalty Avoidance Percentage (all plays)
# Target Prevention Percentage Over Expectation (TPOE) - A

routes['pred_throw_no_defender'] = np.where(routes['sample']=='1H', model_throw_no_defender_2h.predict_proba(routes), model_throw_no_defender_1h.predict_proba(routes))
routes['pred_throw'] = np.where(routes['sample']=='1H', model_throw_2h.predict_proba(routes), model_throw_1h.predict_proba(routes))
routes['pred_arrival'] = np.where(routes['sample']=='1H', model_arrive_2h.predict_proba(routes), model_arrive_1h.predict_proba(routes))
routes['pred_outcome'] = np.where(routes['sample']=='1H', model_outcome_2h.predict_proba(routes), model_outcome_1h.predict_proba(routes))
routes['pred_target'] = np.where(routes['sample']=='1H', model_target_2h.predict_proba(routes), model_target_1h.predict_proba(routes))
routes['pred_penalty'] = np.where(routes['sample']=='1H', model_penalty_2h.predict_proba(routes), model_penalty_1h.predict_proba(routes))
routes['pred_island'] = np.where(routes['sample']=='1H', model_island_2h.predict(routes), model_island_1h.predict(routes))

routes['pcoe'] = routes['pred_throw'] - routes['pred_throw_no_defender']
routes['tcoe'] = routes['pred_throw'] - routes['pred_throw_no_defender']
routes['troe'] = np.where(routes['pass_arrived'].isnull(), np.nan, routes['pred_arrival'] - routes['pred_throw'])
routes['tdoe'] = np.where((routes['pass_arrived'].isnull()|routes['pass_outcome'].isnull()), np.nan, routes['pred_outcome'] - routes['pred_arrival'])
routes['island'] = routes['second_closest_distance_pass'] - routes['pred_island']
routes['boost'] = routes['pred_throw'] - routes['pred_throw_no_defender'] # need to adjust
routes['paoe'] = routes['penalty'] - routes['pred_penalty']
routes['tpoe'] = routes['targeted'] - routes['pred_target']

results = players.copy()
results['routes_defended'] = results.merge(routes.groupby('closest_defender')['targeted'].count(),
                        how='left',left_on=['nflId'],right_on=['closest_defender'],validate='many_to_one')['targeted'].values
results['routes_targeted'] = results.merge(routes.groupby('closest_defender')['targeted'].sum(),
                        how='left',left_on=['nflId'],right_on=['closest_defender'],validate='many_to_one')['targeted'].values
results['second_closest'] = results.merge(routes.groupby('second_closest_defender')['targeted'].sum(),
                        how='left',left_on=['nflId'],right_on=['second_closest_defender'],validate='many_to_one')['targeted'].values
results['team'] = results.merge(routes[['closest_defender','def_team_name']].drop_duplicates(subset=['closest_defender']),
                        how='left',left_on=['nflId'],right_on=['closest_defender'],validate='many_to_one')['def_team_name'].values

results = results.dropna(subset=['routes_defended','routes_targeted','second_closest'])
for c in ['routes_defended','routes_targeted','second_closest']:
    results[c] = results[c].astype(int)

def add_stat(stat, targets_only, closest='closest_defender'):
    if targets_only:
        results[stat] = results[['nflId']].merge(routes[routes.targeted==1].groupby([closest])[stat].mean(),
                        how='left',left_on=['nflId'],right_on=[closest],validate='many_to_one')[stat].round(3).values
    else:
        results[stat] = results[['nflId']].merge(routes.groupby([closest])[stat].mean(),
                        how='left',left_on=['nflId'],right_on=[closest],validate='many_to_one')[stat].round(3).values

add_stat('pcoe', False)
add_stat('tcoe', True)
add_stat('troe', True)
add_stat('tdoe', True)
add_stat('island', False)
add_stat('paoe', False)
add_stat('tpoe', False)

routes['tcoe_adj'] = routes[['closest_defender']].merge(results,how='left',left_on=['closest_defender'],right_on=['nflId'],validate='many_to_one')['tcoe'].values
routes['boost'] = routes['boost'] - routes['tcoe_adj'].fillna(0)
add_stat('boost', True, closest='second_closest_defender')

results = results.query('routes_defended>0').rename(columns={'displayName':'name'})

football_field = mpimg.imread('../input/nfl2021/images/football_field.jpg')

def show_play(play_id):
    
    data = tracking_data[tracking_data.id==play_id]

    #x, y, marker color, marker sizes and jersey num for creating scatter plot for the first frame
    cond = data['frameId'] == 1
    colors = {'home':'whitesmoke', 'away':'coral', 'football':'brown'}
    size = {'home':300, 'away':300, 'football':50}

    x = data.loc[cond, 'x']
    y = data.loc[cond, 'y']
    c = data.loc[cond, 'team'].map(colors)
    s = data.loc[cond, 'team'].map(size)
    jersey_num = data.loc[cond, 'jerseyNumber'].values
    jersey_num = ['' if np.isnan(num) else str(int(num)) for num in jersey_num] #assigning empty string for nan value from football jersey number
    zipped = list(zip(x, y, jersey_num)) #for looping and creating annotations on the marker

    #s used for x position in plot_title. A simple ax.set_title works but it looks very bad in the animation, hence the complicated code with ax.text and creating two line breaks to keep the title text within the plot.
    #set up the plot
    fig, ax = plt.subplots(figsize=(14,8))
    imgplot = ax.imshow(football_field, extent=[0, 120, 0, 53.3])

    #cap xmin at 0 and xman at 120
    xmin = 0 if ((data['x'].min()-20) < 0 or (data['x'].min() < 0)) else data['x'].min()-20
    xmax = 120 if ((data['x'].max()+20 > 120) or (data['x'].max() > 120)) else data['x'].max()+20
    #xmean for centering title text
    xmean = np.mean([xmin, xmax])
    ax.set(xticks=([]), yticks=([]), xlim=(xmin, xmax), ylim=(-5, 53.3));

    #creating title with two line breaks to ensure title stays within plot
    playId = data['playId'].unique().item() #retrieve playId from data
    gameId = data['gameId'].unique().item() #retrieve gameId from data
    title_cond = (plays['playId'] == playId) & (plays['gameId'] == gameId) #use playId and gameId as filter condition

    title_list = plays.loc[title_cond, 'playDescription'].values.item().split(' ') #save title to list
    insert_index = int(round(len(title_list)/3)) #index position for inserting two line breaks

    for i in [insert_index, insert_index*2]: #insert two line break
        title_list.insert(i, '\n')

    title = ' '.join(title_list) #join title list on ' '
    plot_title = ax.text(x=xmean, y=-2.5, s=title, fontsize=11, ha='center', va='center') #plot title with two line break

    #setting up the legend with white text
    team = ['home', 'away']
    color = list(colors.values())[:2]
    team_color = list(zip(team, color))
    for team, color in team_color:
        ax.scatter([], [], label=team, c=color)

    legend = ax.legend(frameon=False)

    #plot scatter points and annotations
    scatter = ax.scatter(x, y, marker='o', linestyle='None', c=c, s=s, edgecolor='black', linewidths=1.2,)
    annotations = [ax.annotate(val[2], xy=(val[:2]), va='center', ha='center') for val in zipped]

    #create animate function to update scatter plot and annotations
    def animate(frame_id):
        #set new xy postions for previously created scatter plot
        xy = data.loc[data['frameId']==frame_id, ['x', 'y']].to_numpy()
        scatter.set_offsets(xy)

        #set new xy postions for previously created annotations
        if frame_id > 0:
            [annotations[i].set_position(tuple(xy[i])) for i in range(len(xy))]


    #call funcanimation
    anim = FuncAnimation(fig, animate, interval=100, frames=len(data['frameId'].unique())+1)
    plt.rcParams['animation.html'] = 'html5'
    return anim

from sklearn.metrics import roc_curve, roc_auc_score

def roc(df, y_true, y_score, label):        
    false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(df[y_true], df[y_score])
    plt.title('ROC - %s'%label)
    plt.plot(false_positive_rate1, true_positive_rate1)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

### Statistic Correlations
Below shows how well the different statistics correlate with Defensive-EPA (d_epa). While no correlations are strong, we do see consistent correlations across both sub-samples, many of which are positive.

In [ ]:
routes['d_epa'] = -1*routes['epa']
cor = pd.concat([
    routes.query('targeted==1').groupby(['sample'])[['tcoe','troe','tdoe','boost',
    'dsr','d_epa']].corr('spearman').reset_index().query('level_1=="d_epa"').round(3).drop(columns='level_1').set_index('sample'),
    routes.groupby(['sample'])[['pcoe','island',
    'tpoe','paoe','dsr','d_epa']].corr('spearman').reset_index().query('level_1=="d_epa"').round(3).drop(columns='level_1').set_index('sample').drop(columns=['dsr','d_epa'])
        ], axis=1)
cor.index.name = ''
cor.drop_duplicates().T.sort_values('1H',ascending=False)

Below shows how reliable the different statistics are for players across the 1H/2H sub-samples, for players with >=50 passes defended or >=20 targets defended (depending on the statistic) across both sub-sample periods.

In [ ]:
player_correlations = []
for c in ['tcoe','troe','tdoe']:
    df = routes.query('targeted==1').groupby(['sample','closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='closest_defender', columns='sample')[['mean','count']]
    df.columns = ['mean_1h','mean_2h','count_1h','count_2h']
    df = df.query('count_1h>=20 and count_2h>=20')
    player_correlations.append([c, len(df), df[['mean_1h','mean_2h']].corr().values[0][1]])

for c in ['boost']:
    df = routes.query('targeted==1').groupby(['sample','second_closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='second_closest_defender', columns='sample')[['mean','count']]
    df.columns = ['mean_1h','mean_2h','count_1h','count_2h']
    df = df.query('count_1h>=20 and count_2h>=20')
    player_correlations.append([c, len(df), df[['mean_1h','mean_2h']].corr().values[0][1]])
    
for c in ['pcoe','island','paoe','tpoe']:
    df = routes.groupby(['sample','closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='closest_defender', columns='sample')[['mean','count']]
    df.columns = ['mean_1h','mean_2h','count_1h','count_2h']
    df = df.query('count_1h>=50 and count_2h>=50')
    player_correlations.append([c, len(df), df[['mean_1h','mean_2h']].corr().values[0][1]])

pd.DataFrame(player_correlations, columns=['stat', 'n', 'correlation']).set_index('stat').sort_values('correlation',ascending=False)

### Key Observations

1. TROE and TCOE are the two statistics that show up as trending positive in terms of having positive correlation with success and also being sticky. The stickiness would likely improve over a larger sample size.
1. ISLAND is sticky but doesn't correlate with success. This might be due to some players being consistently left on an island, but this alone doesn't guarantee teammate success on the play.
1. PCOE (and TPOE to some degree) has potential as it is sticky. It is worth noting these metrics are measuring the correlation of a single route being covered and success of the overall play, therefore they are likely more correlated with success than the numbers above would indicate. Further research is warranted to determine whether the average value across routes improves success correlation. 
1. TDOE seems to have correlation with play success but does not appear to be sticky. This may be due to "disruption" being more luck than skill (i.e. inaccurate pass outcomes might be causing correlation with success but hurting stickiness). It would be interesting to see how evaluation might change with changes to what features are considered (i.e. remove features like football location/speed that correlate with quarterback accuracy/skill).

### Top Ranked Players in TROE

In [ ]:
top = results.query('routes_defended >= 100').copy()
for c in ['troe','tcoe','pcoe','tdoe','island']:
    top['%s_rank'%c] = top[c].rank(method='min',ascending=False).astype(int)
for c in ['tpoe']:
    top['%s_rank'%c] = top[c].rank(method='min',ascending=True).astype(int)
top.sort_values('troe_rank',ascending=True)[['name','team',
    'position','routes_defended','routes_targeted','pcoe_rank','tcoe_rank','troe_rank','tpoe_rank','island_rank']].set_index('name').head(30)

Eddie Jackson (#39) is one player who stands out in both TROE and TCOE and we can see why on the play below. At the time of the pass being thrown the expected outcome was 42% probability of the defense winning the play. However Jackson's ***quick reactions*** increased the probability up to 74%.

In [ ]:
show_play(2018112200424)

See the Appendix for additional results/details/analysis.

---

## 6. Appendix

### A.1. Model Features

***Base Features***
1. **Situation**
 * down
 * yards to go
 * home/away
 * absolute yard line number
1. **Personnel**
 * offense formation
 * defenders in the box
 * number of pass rushers
 * number of receiver routes
1. **Pass details**
 * location of ball time of throw
 * time to pass (proxy for quarterback pressure)
1. **Receiver route details**
 * route
 * receiver position
 * location of receiver at snap
 * location of receiver at time of throw
 * air yards
1. **Receiver movement metrics**
 * speed/acceleration/orientation/direction at time of throw
 * max/avg speed during route (between pass and throw)
 * max acceleration during route
 * distance traveled during route

***Optional Features (varies by model whether used)***

6. **Closest defender metrics (at time of throw)**
 * location of receiver at time of throw
 * speed/acceleration/orientation/direction at time of throw
 * max/avg speed during route (between pass and throw)
 * max acceleration during route
 * distance from receiver at throw
1. **Pass arrival metrics (all at time of arrival)**
 * location/speed/acceleration of football
 * location/speed/acceleration/orientation/direction of closest defender (from time of throw)
 * location/speed/acceleration/orientation/direction of receiver
 * distance of closest defender
1. **Pass outcome metrics (all at time of pass outcome)**
 * location/speed/acceleration of football
 * location/speed/acceleration/orientation/direction of closest defender (from time of throw)
 * location/speed/acceleration/orientation/direction of receiver
 * distance of closest defender

### A.2. Feature Importance
Below we can see which features are most predictive across each model. Note that this is based on 1H model variations and the [permutation importance](https://auto.gluon.ai/api/autogluon.task.html#autogluon.tabular.TabularPredictor.feature_importance) method using 2H as test data.

**Pass-Thrown-No-Defender - Expected Defensive-Success-Rate**

In [ ]:
dv = 'dsr'
model_throw_no_defender_1h.feature_importance(train_2h[base_features+[dv]]).head(10)

**Pass-Thrown - Expected Defensive-Success-Rate**

In [ ]:
model_throw_1h.feature_importance(train_2h[base_features+defender_throw_features+[dv]]).head(10)

**Pass-Arrived - Expected Defensive-Success-Rate**

In [ ]:
model_arrive_1h.feature_importance(train_2h[base_features+defender_throw_features+defender_arrival_features+[dv]]).head(10)

**Pass-Finished - Expected Defensive-Success-Rate**

In [ ]:
model_outcome_1h.feature_importance(train_2h[base_features+defender_throw_features+defender_arrival_features+defender_outcome_features+[dv]]).head(10)

**Expected Target Percentage**

In [ ]:
dv = 'targeted'
model_target_1h.feature_importance(train_2h_all[base_features+[dv]]).head(10)

**Expected Penalty Percentage**

In [ ]:
dv = 'penalty'
model_penalty_1h.feature_importance(train_2h_all[base_features+[dv]], subsample_size=2000).head(10)

**Expected Closest Help Defender Yards**

In [ ]:
dv = 'second_closest_distance_pass'
model_island_1h.feature_importance(train_2h_all[base_features+[dv]]).head(10)

### A.3. Receiver Operating Characteristic (ROC)

Below we can see ROC curves for each of the classification models. As expected, the models get more and more accurate at predicting DSR as additional context is added (no defender -> throw -> arrive -> outcome). We can also see that the target prediction model performs well (i.e. high AUC). On the other hand, the penalty prediction model performs poorly (likely to do such extreme class imbalance).

In [ ]:
roc(routes.query('targeted==1'), 'dsr','pred_throw_no_defender', 'Pass-Thrown-No-Defender (AUC=%s)'%round(model_throw_no_defender_1h.evaluate(train_2h, silent=True),3))

roc(routes.query('targeted==1'), 'dsr','pred_throw', 'Pass-Thrown (AUC=%s)'%round(model_throw_1h.evaluate(train_2h, silent=True),3))

roc(routes.query('targeted==1'), 'dsr','pred_arrival', 'Pass-Arrived (AUC=%s)'%round(model_arrive_1h.evaluate(train_2h, silent=True),3))

roc(routes.query('targeted==1'), 'dsr','pred_outcome', 'Pass-Finished (AUC=%s)'%round(model_outcome_1h.evaluate(train_2h, silent=True),3))

roc(routes, 'targeted','pred_target', 'Expected Target Percentage (AUC=%s)'%round(model_target_1h.evaluate(train_2h_all, silent=True),3))

roc(routes, 'penalty','pred_penalty', 'Expected Penalty Percentage (AUC=%s)'%round(model_penalty_1h.evaluate(train_2h_all, silent=True),3))

### A.4. Model Considerations
* While overall EPA was considered, we chose DSR as the model target to (1) improve stability - success rate is less susceptible to bias from outlier plays<sub>2</sub> and (2) eliminate bias from post pass outcome events that cause large swings in EPA (e.g. broken tackle touchdown, fumble for defensive touchdown, etc.).
* We chose to include sacks as a "time of throw" event in order to avoid throwing out data when no pass was attempted on the play as this is partially due to strong pass coverage.
* Quarterback and receiver skill are not explicitly factored in by the models. This helps avoid challenges with out of sample prediction and/or small sample sizes. That being said, quarterback throw difficulty is considered based on "pass details" data and receiver coverage difficulty is explicitly factored in through "receiver movement metrics" (i.e. speed, acceleration, etc.). Below we will see that quarterback and receiver skill is "averaged out" when evaluating defensive players and limits exposure of specific offensive player skill from impacting defender player evaluation (i.e. defenders are compared against expected DSR in a given situation instead of actual EPA).
* While the intent of not having defender features known by some of the models is to keep the model unbiased by defender player skill, it is worth noting that defenders can have some impact on receiver movement metrics (jamming at the line, slowing down, redirecting, etc.). This is something to be aware of and worth revisiting.

<sup>2</sup> *I haven't confirmed success rate is more sticky season-to-season but [this post](https://www.footballperspective.com/rushing-epa-and-yards-per-carry/) suggests this is true for rushing EPA.*

### A.5. Possible Extensions

Some possible extensions of this work might include:
* This analysis was only looking at one season worth of data (and only a sub-sample of the data for modeling). It would be interesting to see how looking across seasons might change results. Using a larger data sample might open up an opportunity to segment stats further (i.e. by Man vs. Zone coverage) to learn even more about players. 
* Given the robust AutoML framework used in this analysis there was no effort done to prune non-predictive features. It would be useful to explore trimming the feature set down into only the most predictive features. The feature set for each model should also be holistically re-evaluated, as certain features could in theory lead to leakage in terms of things that are/aren't in control of the player. For example, things like player orientation/ball speed can proxy for things like "poorly thrown ball" or "decoy route" which are out of control of the defender and could in theory make a statistic less sticky.
* The decision to use “second closest defender" to capture team effects is easy to implement but likely isn't optimal. Further research should be done to determine whether there exists a better heuristic (i.e. "within X yards"). Probing coaches and players for insight on this topic would likely be revealing.
* Given the small sample sizes we could consider using [Empirical Bayes](https://tothemean.com/2020/09/06/empirical-bayes.html) to regress players to the mean and likely improve stability.

### A.6. Success Correlation

In [ ]:
for c in ['tcoe','troe','tdoe','boost']:
    sns.regplot(routes.query('targeted==1')[c], routes.query('targeted==1')['d_epa']).set_title('Defensive EPA vs. %s Correlation'%c.upper())
    plt.show()
for c in ['pcoe','island','paoe','tpoe']:
    sns.regplot(routes[c], routes['d_epa']).set_title('Defensive EPA vs. %s Correlation'%c.upper())
    plt.show()

### A.7. Player Correlation

In [ ]:
for c in ['tcoe','troe','tdoe']:
    df = routes.query('targeted==1').groupby(['sample','closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='closest_defender', columns='sample')[['mean','count']]
    df.columns = ['1h','2h','count_1h','count_2h']
    df = df.query('count_1h>=20 and count_2h>=20')
    sns.regplot(df['1h'], df['2h']).set_title(c)
    plt.show()

for c in ['boost']:
    df = routes.query('targeted==1').groupby(['sample','second_closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='second_closest_defender', columns='sample')[['mean','count']]
    df.columns = ['1h','2h','count_1h','count_2h']
    df = df.query('count_1h>=20 and count_2h>=20')
    sns.regplot(df['1h'], df['2h']).set_title(c)
    plt.show()
    
for c in ['pcoe','island','paoe','tpoe']:
    df = routes.groupby(['sample','closest_defender'])[c].agg({'count','mean'}).reset_index()
    df = df.pivot(index='closest_defender', columns='sample')[['mean','count']]
    df.columns = ['1h','2h','count_1h','count_2h']
    df = df.query('count_1h>=50 and count_2h>=50')
    sns.regplot(df['1h'], df['2h']).set_title(c)
    plt.show()

### A.8. Additional Results

#### Top Ranked Players in TCOE

In [ ]:
top.sort_values('tcoe_rank',ascending=True)[['name','team',
    'position','routes_defended','routes_targeted','pcoe_rank','tcoe_rank','troe_rank','tpoe_rank','island_rank']].set_index('name').head(25)

Quinton Dunbar (#23) shows up as one of the top corners in TCOE. The play below was projected to have a 44% probability of the defense winning at time of throw based on the "Pass-Thrown-No-Defender" model. However, with the "Dunbar's ***strong coverage*** increased that probability to 74% based on the "Pass-Thrown" model.

In [ ]:
show_play(20181122012283.)

#### Top Ranked Players in PCOE

In [ ]:
top.sort_values('pcoe_rank',ascending=True)[['name','team',
    'position','routes_defended','routes_targeted','pcoe_rank','tcoe_rank','troe_rank','tpoe_rank','island_rank']].set_index('name').head(25)

#### Top Ranked Players in TPOE

In [ ]:
top.sort_values('tpoe_rank',ascending=True)[['name','team',
    'position','routes_defended','routes_targeted','pcoe_rank','tcoe_rank','troe_rank','tpoe_rank','island_rank']].set_index('name').head(25)

#### Top Ranked Players in ISLAND

In [ ]:
top.sort_values('island_rank',ascending=True)[['name','team',
    'position','routes_defended','routes_targeted','pcoe_rank','tcoe_rank','troe_rank','tpoe_rank','island_rank']].set_index('name').head(25)

#### Patriots Team View

In [ ]:
results.query('team=="NE" and routes_defended>20')[['name','team',
    'position','routes_defended','routes_targeted','pcoe','tcoe','troe','tpoe','island']].set_index('name').sort_values('pcoe',ascending=False)